### YOUTUBE 리뷰 수집

In [1]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd
import time
from tqdm import tqdm as tq
from bs4 import BeautifulSoup as bs
import os

options = Options()
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함


In [10]:
def review_collection():
    # YOUTUBE 댓글 수집 기능
    s = Service(ChromeDriverManager().install()) #"127.0.6533.119" 명시하면 강제로 다운 가능
    driver = wb.Chrome(service = s, options=options)
    url = "https://www.youtube.com/watch?v=91TNlSs9S_0"
    driver.get(url)

    driver.maximize_window()
    time.sleep(2)

    # 스크롤을 내려야, 리뷰 로딩해서 출력
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.END)
    time.sleep(2)

    # 기존의 리뷰 개수!
    soup = bs(driver.page_source,"lxml")
    review_cnt = soup.select("yt-attributed-string#content-text > span")

    old_cnt = len(review_cnt)
    while True: # 스크롤 하강
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.END)
        time.sleep(2)
        soup = bs(driver.page_source, "lxml")
        review_cnt = soup.select("yt-attributed-string#content-text > span")

        if len(review_cnt) >= 100:
            print("리뷰 100건으로 종료")
            break

        new_cnt = len(review_cnt)
        if old_cnt == new_cnt:
            break
        old_cnt = new_cnt

        # 작성자 수집
        # 리뷰 수집
        writer = driver.find_elements(By.CSS_SELECTOR, "#author-text > span ")
        review = driver.find_elements(By.CSS_SELECTOR, "yt-attributed-string#content-text > span")

        # 주인장 리뷰 삭제(분석에 의미가 없음)
        del writer[0]
        del review[0]

        id_list = [id.text for id in writer]
        re_list = [re.text for re in review]

        dict = {
            "ID" : id_list,
            "리뷰" : re_list
        }

        df = pd.DataFrame(dict)
        df.to_csv("youtube_review.csv", encoding = "UTF-8")
review_collection()

리뷰 100건으로 종료


## 